In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [3]:
class_names = os.listdir('C:/Users/ritti/OneDrive/Desktop/MT2022090,MT2022162/dogs-vs-cats/train')
len(class_names)

2

In [6]:
images = [] #Get path to all images and save them in a list
labels = [] # labels of corresponding images
class_id = 0

for i in class_names:
    dir=os.path.join('C:/Users/ritti/OneDrive/Desktop/MT2022090,MT2022162/dogs-vs-cats/train',i)
    x = [os.path.join(dir, f) for f in os.listdir(dir)]
    images+= x
    labels+=[class_id]*len(x)
    class_id+=1

In [8]:
len(images)

25000

In [17]:
import random
D = []
for i in range(len(images)):
    D.append([images[i],labels[i]])
dataset = D
random.shuffle(dataset)
train = dataset[:20000]
test = dataset[20000:]

In [16]:
len(train)

20000

In [18]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
features = models.alexnet().features  # get the feature extraction layers
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
flatten = torch.nn.Flatten()# create a flatten layer
model = torch.nn.Sequential(features, flatten).to(device)  # create a new model with the flatten layer
summary(model)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0

In [19]:
from tqdm import tqdm
import cv2
def extract_features(train):
    features = []
    labels = []
    for i in tqdm(train,desc='loading...'):
        im = cv2.imread(i[0])
        im = im.astype('float32')
        im = cv2.resize(im, (224,224))
        im = torch.from_numpy(im)
        im = im.to(device)
        im = im.unsqueeze(0) # adding an extra dimension at index 0 which refers to the batch size =1
        im = im.permute(0, 3, 1, 2) # reorder the dimensions of the input tensor from [batch_size, height, width, num_channels] to [batch_size, num_channels, height, width]. 
        with torch.no_grad():
            outputs = model(im)
            features.append(outputs.cpu().numpy()[0])
            labels.append(i[1])
    return np.array(features), np.array(labels)

In [20]:
x_train, y_train = extract_features(train)
x_test, y_test = extract_features(test)

loading...: 100%|██████████| 5000/5000 [02:47<00:00, 29.89it/s]


In [21]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(20000, 9216)
(20000,)
(5000, 9216)
(5000,)


In [23]:
from sklearn import svm
clf = svm.SVC(C=1.8, random_state=42)
clf.fit(x_train, y_train)

SVC(C=1.8, random_state=42)

In [24]:
from sklearn.metrics import accuracy_score
pred_labels = clf.predict(x_test)
accuracy = accuracy_score(y_test, pred_labels)
print('Accuracy:', accuracy)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred_labels)
print(cm)

from sklearn.metrics import f1_score
f1score = f1_score(y_test, pred_labels, average='weighted')
print("F1 score: {:.2f}".format(f1score))

Accuracy: 0.7396
[[1848  619]
 [ 683 1850]]
F1 score: 0.74
